In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
dept=6
source="PA_PS"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, specialite, source, time, accessibilite_exp)

21 10 PA_PS 30 -0.12


In [3]:
d="an.dept_id" if dept is None else str(dept) # Non utilisé
sources: dict[str, str] = {
    "PS": f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
""",
    "PA": f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
""",
    "RPPS": f"""
select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and personne.date_maj > '20{year}-01-01' and date_effet < '20{year}-01-01'
and an.dept_id={dept}
group by personne.id, an.id, i.id
""",
    "PA_PS": "Not implemented",
}
sql = sources[source]
print(sql)

Not implemented


In [4]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [5]:
def get_pa(year, specialite):
    sql=f"""
    select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)


In [6]:
def get_pa_outer_ps(year, specialite):
    sql=f"""
with ps2 as (
    select ps.id, ps.key, an.lon, an.lat from ps
    join tarif t on t.ps_id = ps.id
    join tarif_date_source tds on tds.tarif_id=t.id
    join date_source ds on ds.id=tds.date_source_id
    join cabinet c on t.cabinet_id=c.id
    join adresse_raw ar on c.adresse_raw_id=ar.id
    join adresse_norm an on ar.adresse_norm_id=an.id
    join specialite_profession sp on sp.profession_id=t.profession_id
    --join specialite s on s.id=sp.specialite_id
    where sp.specialite_id={specialite}
    and ds.annee={year}
)
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, coalesce(an.lon,ps2.lon) as lon, coalesce(an.lat,ps2.lat) as lat
from personne_activite pa
left outer join ps2 on ps2.key=pa.inpp
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and pands.date_source_id/100={year}
and an.dept_id={dept}
and coalesce(an.lon,ps2.lon) is not null
group by pa.id, an.id, i.id, coalesce(an.lon,ps2.lon), coalesce(an.lat,ps2.lat)
"""
    print(f"Quering PA-Outer-PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [7]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA_PS":
        return get_pa_outer_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [8]:
# df = pd.read_sql(sql, config.connection_string)
# df

In [9]:
df = get_by_source(year, specialite, source)
df

Quering PA-Outer-PS for year 21 and specialite 10


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037
1,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167
2,4673,PIONNEAU,AUDREY,6,272923,61120000,7.003135,43.673439
3,11222,LANSON,FREDERIC,6,2335,61230105,7.192090,43.671030
4,16807,DITTLOT,PIERRE,6,75209,60882601,7.241556,43.689933
...,...,...,...,...,...,...,...,...
701,1529201,BERGONZO,CHRISTIAN,6,3146,60330103,7.192359,43.773162
702,1530155,GUERIN,ALEXANDRE,6,84193,60882601,7.237917,43.687275
703,1534638,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533
704,1540045,CORALLINI TEISSIER,MARIE MADELEINE,6,63059,60881703,7.267766,43.706909


In [10]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 670
Nb unique PS 706


In [11]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.899433
2    0.096317
3    0.004249
Name: proportion, dtype: float64

In [12]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037,3568_43.570037_7.008846,1,1.0,3.0
1,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167,4008_43.588167_7.040872,1,1.0,2.0
2,4673,PIONNEAU,AUDREY,6,272923,61120000,7.003135,43.673439,4673_43.673439_7.003135,1,1.0,5.0
3,11222,LANSON,FREDERIC,6,2335,61230105,7.192090,43.671030,11222_43.67103_7.19209,1,1.0,3.0
4,16807,DITTLOT,PIERRE,6,75209,60882601,7.241556,43.689933,16807_43.689933_7.241556,1,1.0,8.5
5,16864,LAPICQUE,PIERRE,6,1590,60480101,7.315379,43.810132,16864_43.810132_7.315379,1,1.0,4.0
6,23645,BURTE,CAROLE,6,2491,60290111,7.021753,43.552328,23645_43.552328_7.021753,1,1.0,15.0
7,24447,CALEMCZUK,GAELLE,6,77494,61490102,7.313438,43.743446,24447_43.743446_7.313438,1,1.0,10.0
8,30483,GARIBBO,RODOLPHE,6,81978,61040105,7.482690,43.764934,30483_43.764934_7.48269,1,1.0,2.0
9,37163,MOREL MANDRINO,PATRICIA,6,2852,60883801,7.277891,43.733607,37163_43.733607_7.277891,2,0.5,7.5


In [13]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
703,1534638,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,1534638_43.604533_7.092388,1,1.0,1.0
634,1388268,COQUELET MOULINOUX,ANNE,6,277049,60040105,7.115366,43.593250,1388268_43.59325_7.115366,1,1.0,1.0
571,1244808,COGGIA DEVILERS,CAROLE,6,63070,60040107,7.122697,43.600420,1244808_43.60042_7.122697,1,1.0,1.0
152,318705,CANAS,PAUL,6,75227,60040109,7.097858,43.594409,318705_43.594409_7.097858,1,1.0,2.0
138,300440,PREVOST,CHARLOTTE,6,2366,60040114,7.121534,43.581389,300440_43.581389_7.121534,1,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
472,1006859,BARTOLI,LAURENT,6,2426,61610102,7.123851,43.664508,1006859_43.664508_7.123851,1,1.0,1.0
538,1171597,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,1171597_43.658047_7.123625,1,1.0,1.0
95,226731,CHEMOUNI,KARINE,6,84186,61610104,7.142204,43.642154,226731_43.642154_7.142204,1,1.0,1.0
197,396950,LEBRUN CHAUVOT,EMILIE,6,271634,61610105,7.141885,43.640635,396950_43.640635_7.141885,1,1.0,3.0


In [14]:
def get_iris_matrix(time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m

# iris_matrix = get_iris_matrix(time_type)
iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time,iris
0,40080000,60530000,17,22,60530000
1,40080000,60990000,21,24,60990000
2,40390000,61540000,24,30,61540000
3,40420000,60530000,11,19,60530000
4,40420000,60710000,22,30,60710000
...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103
117765,831380000,60950101,18,19,60950101
117766,831380000,60950102,19,22,60950102
117767,831380000,60950103,18,20,60950103


In [15]:
def get_pop_iris():
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,60120105,060120105,06012,A,3136.830308,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,831.040803,391.639115,351.412985,194.786785
1,60500000,060500000,06050,C,526.000000,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,145.187085,83.001180,65.334467,26.630014
2,60720000,060720000,06072,C,142.000000,0.000000,6.761905,4.829932,4.829932,3.863946,21.251701,27.047619,27.047619,31.877551,14.489796
3,61050101,061050101,06105,A,4007.649525,96.881691,145.234138,320.060918,454.509675,180.143151,475.809676,953.212078,561.152910,592.431015,228.214272
4,60150000,060150000,06015,C,1227.000000,35.622418,38.520557,52.247192,81.918133,62.907751,196.126186,266.670404,226.738245,194.355336,71.893777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60881606,060881606,06088,A,2621.558372,56.982735,81.433187,104.491854,148.417994,461.133207,399.230815,389.919891,334.254784,416.753195,228.940710
496,60881701,060881701,06088,A,1708.308379,24.463401,43.859704,85.003020,126.388729,282.367384,250.163180,355.063504,193.244615,223.033528,124.721315
497,60881804,060881804,06088,A,2893.228431,85.043040,83.274932,124.543590,225.112876,294.722303,673.422185,570.114686,285.987223,350.796724,200.210873
498,60881805,060881805,06088,A,2156.226647,65.721430,54.233925,76.225674,153.780470,257.227060,430.652161,329.042912,253.515770,331.504051,204.323195


In [16]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,40080000,60530000,17,22,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
1,40080000,60990000,21,24,60990000,060990000,06099,C,1813.000000,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272
2,40390000,61540000,24,30,61540000,061540000,06154,C,495.000000,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879
3,40420000,60530000,11,19,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
4,40420000,60710000,22,30,60710000,060710000,06071,C,584.000000,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,060850103,06085,A,2603.604864,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,479.908067,393.484524,562.460591,247.925725
117765,831380000,60950101,18,19,60950101,060950101,06095,A,3223.464405,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,685.139541,418.753123,589.352449,264.071324
117766,831380000,60950102,19,22,60950102,060950102,06095,A,2563.685976,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,529.572959,354.290116,427.490829,177.066411
117767,831380000,60950103,18,20,60950103,060950103,06095,A,2468.849619,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,537.700844,388.050677,528.083677,185.973993


In [17]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

# apl["time"] = apl["time"].replace(0, 0.5)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,40080000,60530000,17,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361
1,40080000,60990000,21,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
2,40390000,61540000,24,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324
3,40420000,60530000,11,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284
4,40420000,60710000,22,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324
5,40420000,60990000,17,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
6,40420000,61240000,15,30,61240000,061240000,06124,C,52.0,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,4.262295,7.672131,23.016393,5.967213,0.027324
7,40430000,60020000,15,20,60020000,060020000,06002,C,51.0,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,12.277778,12.277778,14.166667,2.833333,0.090718
8,40430000,60240000,8,11,60240000,060240000,06024,C,184.0,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135
9,40430000,60450000,18,23,60450000,060450000,06045,C,76.0,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.063292


In [18]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
105136,61290000,60730000,14,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029
105137,61290000,60800000,10,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325
105138,61290000,60820000,23,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324
105139,61290000,61020000,13,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460
105140,61290000,61100000,12,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284
105141,61290000,61110000,10,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029
105142,61290000,61290000,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000
105143,61290000,61460000,18,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292
105144,61290000,61530000,14,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460


In [19]:
cols = [col for col in apl.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [20]:
sql = f"""
select o.* from apl.overrepresentation o
where o.year={year} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,1.166949,0.539683,0.402415,0.420643,0.708842,0.929717,0.886061,1.074038,1.491386,2.405645


In [21]:
weights = over.values[0,2:]
weights

array([1.1669492 , 0.53968312, 0.40241483, 0.42064349, 0.70884153,
       0.92971694, 0.88606123, 1.07403771, 1.49138583, 2.40564513])

In [22]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
# if specialite == 5:
#     apl["pop_gp"] /= 2
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361,167.589023
1,40080000,60990000,21,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135,1899.726451
2,40390000,61540000,24,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324,537.620953
3,40420000,60530000,11,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284,167.589023
4,40420000,60710000,22,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324,677.528167


In [23]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
105136,61290000,60730000,14,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029,671.865441
105137,61290000,60800000,10,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325,122.207793
105138,61290000,60820000,23,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324,133.386689
105139,61290000,61020000,13,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460,152.433762
105140,61290000,61100000,12,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284,126.126181
105141,61290000,61110000,10,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029,139.759178
105142,61290000,61290000,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000,317.755204
105143,61290000,61460000,18,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292,144.674411
105144,61290000,61530000,14,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460,878.583120


In [24]:
# apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
# apl = apl.drop(cols, axis=1)
apl = apl[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,40080000,60530000,17,22,0.071361,167.589023,143.0,C
1,40080000,60990000,21,24,0.056135,1899.726451,1813.0,C
2,40390000,61540000,24,30,0.027324,537.620953,495.0,C
3,40420000,60530000,11,19,0.102284,167.589023,143.0,C
4,40420000,60710000,22,30,0.027324,677.528167,584.0,C


In [25]:
apl["iris"] = apl["iris1"].copy()
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,40080000,60530000,17,22,0.071361,167.589023,143.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,0.056135,1899.726451,1813.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,0.027324,537.620953,495.000000,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,0.102284,167.589023,143.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,0.027324,677.528167,584.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,0.030807,2922.034770,2603.604864,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,0.102284,3456.615760,3223.464405,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,0.071361,2669.222378,2563.685976,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,0.090718,2679.719687,2468.849619,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  1.        ,  2.        ,  8.        , 15.        ,
        4.        ,  9.        ,  3.        ,  7.        ,  5.5       ,
        6.        ,  5.        ,  0.5       ,  1.5       ,  2.5       ,
        6.5       ,  9.5       ,  2.33333333,  4.5       , 11.        ,
        8.5       , 19.        ,  7.33333333,  7.5       ,  3.5       ,
       10.        ])

In [27]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40080000,60530000,17,22,0.071361,167.589023,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0
1,40080000,60990000,21,24,0.056135,1899.726451,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,106.640694,118.600059,0.0
2,40390000,61540000,24,30,0.027324,537.620953,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.689806,14.689806,0.0
3,40420000,60530000,11,19,0.102284,167.589023,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0
4,40420000,60710000,22,30,0.027324,677.528167,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.512592,144.237361,0.0


In [28]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
76,60010000,60010000,C,99.275089,0.0,0.000000,295.085798,99.275089,99.275089,93.000000
87,60020000,60020000,C,59.473302,0.0,0.000000,254.968450,59.473302,59.473302,51.000000
98,60030000,60030000,C,621.725124,0.0,0.000000,1230.503600,621.725124,621.725124,640.000000
112,60040101,60040101,C,1725.263970,0.0,0.000000,190008.358917,1725.263970,1725.263970,1763.364267
477,60040102,60040102,A,2534.690523,0.0,0.000000,151833.796108,2534.690523,2534.690523,2534.265666
...,...,...,...,...,...,...,...,...,...,...
115797,61610105,61610105,A,3977.633694,3.0,1.463308,205014.939812,3977.633694,3977.633694,3584.453834
116147,61610106,61610106,A,3823.078668,0.0,0.000000,148752.497762,3823.078668,3823.078668,3539.143531
116501,61610107,61610107,A,2781.820360,0.0,0.000000,162704.435974,2781.820360,2781.820360,2766.654836
116506,61620000,61620000,C,848.449890,0.0,0.000000,2019.206588,848.449890,848.449890,719.000000


In [29]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,40080000,60530000,17,22,0.071361,167.589023,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023,143.0
1,40420000,60530000,11,19,0.102284,167.589023,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023,143.0
2,40760000,60530000,15,19,0.102284,167.589023,143.0,C,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,764.977605,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023,143.0
3,40900000,60530000,22,28,0.034735,167.589023,143.0,C,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.821248,57.728846,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023,143.0
4,41740000,60530000,14,20,0.090718,167.589023,143.0,C,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.203333,150.770225,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023,143.0


In [30]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,40080000,60530000,17,22,0.071361,167.589023,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0,60530000,C,167.589023,0.0,0.000000,396.976547,167.589023,167.589023,143.0,0.000000,2.481017
12,40080000,60990000,21,24,0.056135,1899.726451,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,106.640694,118.600059,0.0,60990000,C,1899.726451,1.0,44.197516,2262.570591,1899.726451,1899.726451,1813.0,2.481017,2.481017
33,40390000,61540000,24,30,0.027324,537.620953,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.689806,14.689806,0.0,61540000,C,537.620953,0.0,0.000000,972.416215,537.620953,537.620953,495.0,0.000000,0.000000
1,40420000,60530000,11,19,0.102284,167.589023,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0,60530000,C,167.589023,0.0,0.000000,396.976547,167.589023,167.589023,143.0,0.000000,2.481017
51,40420000,60710000,22,30,0.027324,677.528167,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.512592,144.237361,0.0,60710000,C,677.528167,0.0,0.000000,1209.227565,677.528167,677.528167,584.0,0.000000,2.481017


In [31]:
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
385,60010000,60010000,0,0,1.0,99.275089,93.000000,C,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.275089,295.085798,0.000000,60010000,C,99.275089,0.0,0.000000,295.085798,99.275089,99.275089,93.000000,0.000000,16.484119
157,60020000,60020000,0,0,1.0,59.473302,51.000000,C,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,59.473302,254.968450,0.000000,60020000,C,59.473302,0.0,0.000000,254.968450,59.473302,59.473302,51.000000,0.000000,0.000000
242,60030000,60030000,0,0,1.0,621.725124,640.000000,C,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,621.725124,1230.503600,0.000000,60030000,C,621.725124,0.0,0.000000,1230.503600,621.725124,621.725124,640.000000,0.000000,0.000000
74748,60040101,60040101,0,0,1.0,1725.263970,1763.364267,C,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1725.263970,190008.358917,0.000000,60040101,C,1725.263970,0.0,0.000000,190008.358917,1725.263970,1725.263970,1763.364267,0.000000,63.925753
74398,60040102,60040102,0,0,1.0,2534.690523,2534.265666,A,60040102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2534.690523,151833.796108,0.000000,60040102,A,2534.690523,0.0,0.000000,151833.796108,2534.690523,2534.690523,2534.265666,0.000000,50.978028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39136,61610105,61610105,0,0,1.0,3977.633694,3584.453834,A,61610105,396950.0,LEBRUN CHAUVOT,EMILIE,6.0,271634.0,7.141885,43.640635,396950_43.640635_7.141885,1.0,1.0,3.0,3977.633694,205014.939812,1.463308,61610105,A,3977.633694,3.0,1.463308,205014.939812,3977.633694,3977.633694,3584.453834,1.463308,70.732168
38772,61610106,61610106,0,0,1.0,3823.078668,3539.143531,A,61610106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3823.078668,148752.497762,0.000000,61610106,A,3823.078668,0.0,0.000000,148752.497762,3823.078668,3823.078668,3539.143531,0.000000,49.089657
38423,61610107,61610107,0,0,1.0,2781.820360,2766.654836,A,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2781.820360,162704.435974,0.000000,61610107,A,2781.820360,0.0,0.000000,162704.435974,2781.820360,2781.820360,2766.654836,0.000000,55.213835
116175,61620000,61620000,0,0,1.0,848.449890,719.000000,C,61620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,848.449890,2019.206588,0.000000,61620000,C,848.449890,0.0,0.000000,2019.206588,848.449890,848.449890,719.000000,0.000000,21.678033


In [32]:
apl["apl"].describe() #mean=95 vs 98, median=101 vs 110, after damir changes mean=95 median=99

count    500.000000
mean      53.153688
std       32.897584
min        0.000000
25%       32.791691
50%       49.122815
75%       74.721851
max      232.577213
Name: apl, dtype: float64

In [33]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl


In [34]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
385,60010000,C,0.0,16.484119,0.0,1.0,99.275089,295.085798,0.0,99.275089,93.000000
157,60020000,C,0.0,0.000000,0.0,1.0,59.473302,254.968450,0.0,59.473302,51.000000
242,60030000,C,0.0,0.000000,0.0,1.0,621.725124,1230.503600,0.0,621.725124,640.000000
74748,60040101,C,0.0,63.925753,0.0,1.0,1725.263970,190008.358917,0.0,1725.263970,1763.364267
74398,60040102,A,0.0,50.978028,0.0,1.0,2534.690523,151833.796108,0.0,2534.690523,2534.265666


In [35]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2021 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [36]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,2021,10,60010000,060010000,C,0.0,16.484119,0.000000,1.0,0.000000,99.275089,295.085798,99.275089,93.000000,Aiglun,6,06001,Aiglun
1,2021,10,60020000,060020000,C,0.0,0.000000,0.000000,1.0,0.000000,59.473302,254.968450,59.473302,51.000000,Amirat,6,06002,Amirat
2,2021,10,60030000,060030000,C,0.0,0.000000,0.000000,1.0,0.000000,621.725124,1230.503600,621.725124,640.000000,Andon,6,06003,Andon
3,2021,10,60040101,060040101,C,0.0,63.925753,0.000000,1.0,0.000000,1725.263970,190008.358917,1725.263970,1763.364267,Les Trois Moulins,6,06004,Antibes
4,2021,10,60040102,060040102,A,0.0,50.978028,0.000000,1.0,0.000000,2534.690523,151833.796108,2534.690523,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021,10,61610105,061610105,A,3.0,70.732168,1.463308,1.0,1.463308,3977.633694,205014.939812,3977.633694,3584.453834,Marina,6,06161,Villeneuve-Loubet
496,2021,10,61610106,061610106,A,0.0,49.089657,0.000000,1.0,0.000000,3823.078668,148752.497762,3823.078668,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
497,2021,10,61610107,061610107,A,0.0,55.213835,0.000000,1.0,0.000000,2781.820360,162704.435974,2781.820360,2766.654836,Plan,6,06161,Villeneuve-Loubet
498,2021,10,61620000,061620000,C,0.0,21.678033,0.000000,1.0,0.000000,848.449890,2019.206588,848.449890,719.000000,La Brigue,6,06162,La Brigue


In [37]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["apl"] > 400]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label


In [38]:
apl.to_csv("apl06.csv", index=False)